In [1]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import time

from scipy.io import wavfile
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read WAV FILE
samp_rate, data = wavfile.read("Song0_KieLoKaz_-_02_-_Trip_to_Ganymed_Kielokaz_ID_363.wav")

In [3]:
mono_channel = data[:,0]

x_train = mono_channel.copy()

# Cut down the data
x_train = x_train[:5120000]

# One hot encode output channels
enc = OneHotEncoder(categories='auto')
enc.fit(x_train.reshape(-1,1))

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [4]:
x_train_onehot = enc.transform(x_train.reshape(-1,1)).toarray()
#y_train_onehot = enc.transform(y_train.reshape(-1,1)).toarray()

In [7]:
x_train_onehot = x_train_onehot.reshape(-1, 1024, 238)

In [8]:
def add_wavenet_layer(input_layer, dilation):
    
    # Layer
    conv_sig = tf.keras.layers.Conv1D(filters=255, kernel_size=2, strides=1, padding="causal",
                                          dilation_rate=dilation, activation='sigmoid')(input_layer)
    conv_tanh = tf.keras.layers.Conv1D(filters=255, kernel_size=2, strides=1, padding="causal",
                                          dilation_rate=dilation, activation='tanh')(input_layer)
    resulting_layer = tf.keras.layers.Multiply()([conv_sig, conv_tanh])

    return(resulting_layer)

In [9]:
model_input = tf.keras.layers.Input(shape=(1024,238))

# Add a single WAVENET layer
out_layer = add_wavenet_layer(model_input, 1)

#Add another 9 (10 are used in the paper - 1024 input span)
for i in range(0):
     out_layer = add_wavenet_layer(out_layer, 2**(i+1))

# Add final output softmax
output_pred = tf.keras.layers.Conv1D(filters=238, kernel_size=1, padding='same', activation='sigmoid')(out_layer)

In [10]:
test_model = tf.keras.Model(model_input, output_pred)

In [11]:
test_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 238)]  0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1024, 255)    121635      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1024, 255)    121635      input_1[0][0]                    
__________________________________________________________________________________________________
multiply (Multiply)             (None, 1024, 255)    0           conv1d[0][0]                     
                                                                 conv1d_1[0][0]               

In [12]:
# Compile and fit
test_model.compile(optimizer='adam', loss='categorical_crossentropy')

test_model.fit(x_train_onehot, x_train_onehot, epochs=1)

Train on 5000 samples
5000/5000 [==============================] - 16s 3ms/sample - loss: 3.9425


In [13]:
29000000*1024*256

7602176000000